# ライブラリ

In [ ]:
# ライブラリインポート
import pandas as pd
from scipy import stats
import random
import numpy as np

# warningsを表示しない
import warnings
warnings.filterwarnings("ignore")

# RCTデータからバイアスのあるデータを作成

In [ ]:
# データ読み込み
email_data = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")
#email_data.head()

# 女性向けのメールが配信されたデータを削除する
male_df = email_data[email_data["segment"] != "Womens E-Mail"].copy()

# 介入を表すtreatment変数を追加
male_df["treatment"] = male_df["segment"].apply(lambda x: 1 if x == "Mens E-Mail" else 0)
#male_df

# バイアスのあるデータを作成

# 購買傾向が一定以上あるユーザーの条件
conditions = (male_df["history"] > 300) | (male_df["recency"] < 6) | (male_df["channel"] == "Multichannel")

# 購買傾向が一定以上のユーザーに重点的にメール配信したかのようなデータを作成
# 以下のユーザをデータから落とす
# ・購買傾向が一定以上かつメール配信されていない
# ・購買傾向が一定未満かつメール配信された
biased_data = pd.concat([
                         
                         # 購買傾向が一定以上のユーザー
                         male_df[(conditions) & (male_df["treatment"] == 0)].sample(frac = 0.5, random_state = 1), 
                         male_df[(conditions) & (male_df["treatment"] == 1)], 
                         
                         # 購買傾向が一定未満のユーザー
                         male_df[(~conditions) & (male_df["treatment"] == 0)], 
                         male_df[(~conditions) & (male_df["treatment"] == 1)].sample(frac = 0.5, random_state = 1)

], axis = 0, ignore_index = True)
biased_data

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
0,8,5) $500 - $750,572.65,1,0,Urban,1,Web,No E-Mail,0,0,0.0,0
1,5,1) $0 - $100,42.38,1,0,Urban,1,Phone,No E-Mail,1,0,0.0,0
2,1,"7) $1,000 +",3003.48,1,1,Urban,1,Phone,No E-Mail,0,0,0.0,0
3,1,5) $500 - $750,662.10,0,1,Urban,1,Web,No E-Mail,0,0,0.0,0
4,5,1) $0 - $100,44.37,0,1,Urban,0,Web,No E-Mail,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31920,12,1) $0 - $100,29.99,1,0,Surburban,1,Web,Mens E-Mail,0,0,0.0,1
31921,6,2) $100 - $200,156.37,1,0,Surburban,0,Web,Mens E-Mail,0,0,0.0,1
31922,11,1) $0 - $100,62.56,0,1,Urban,0,Phone,Mens E-Mail,0,0,0.0,1
31923,11,2) $100 - $200,149.71,1,0,Surburban,1,Phone,Mens E-Mail,0,0,0.0,1


# 3.1 傾向スコアのしくみ

## 3.1.2 傾向スコアの推定

In [ ]:
# ライブラリ
import statsmodels.api as sm
from sklearn.neighbors import NearestNeighbors

In [ ]:
# ロジスティック回帰
y = biased_data["treatment"]
x = sm.add_constant(pd.get_dummies(biased_data[["recency", "history", "channel"]], columns = ["channel"], drop_first = True)) # channelはダミー変数に変換

# Logit
model_logit = sm.Logit(y, x)
model_logit_reg = model_logit.fit()
#print(model_logit_reg.summary().tables[1])

# GLM
#model_glm = sm.GLM(y, x, family = sm.families.Binomial())
#model_glm_reg = model_glm.fit()
#print(model_glm_reg.summary().tables[1])

Optimization terminated successfully.
         Current function value: 0.658837
         Iterations 5


In [ ]:
# 傾向スコア
ps = model_logit_reg.predict(x)
ps

0        0.510515
1        0.525216
2        0.912500
3        0.721648
4        0.524174
           ...   
31920    0.316618
31921    0.510144
31922    0.349282
31923    0.361101
31924    0.338370
Length: 31925, dtype: float64

# 3.2 傾向スコアを利用した効果の推定

## 3.2.1 傾向スコアマッチング

- 要見直し

### 愚直に実装

- [参考]「「効果検証入門」をPythonで書いた」https://qiita.com/nekoumei/items/648726e89d05cba6f432

In [ ]:
# [確認] NearestNeighborsの挙動
d1 = pd.DataFrame({"treatment": [1, 1, 1, 1, 1, 1], "data": [1, 2, 3, 4, 6, 5]})
d1.index = ["a", "b", "c", "d", "e", "f"]
d2 = pd.DataFrame({"treatment": [0, 0, 0, 0, 0], "data": [5,4,3,2,1]})
d2.index = ["A", "B", "C", "D", "E"]

# indicesにはd1のインデックスが入る
neigh = NearestNeighbors(n_neighbors = 1).fit(d1["data"].values.reshape(-1, 1))
distances, indices = neigh.kneighbors(d2["data"].values.reshape(-1, 1))
aftermatch = pd.DataFrame({"distance": distances.reshape(-1), "index": indices.reshape(-1)})
aftermatch

,distance,index
0,0.0,5
1,0.0,3
2,0.0,2
3,0.0,1
4,0.0,0


In [ ]:
# 最近傍マッチングを実施し，マッチング後のデータを返す関数
def get_matched_dfs_using_propensity_score(X, y, random_state = 0):

  # 傾向スコアの計算：ロジスティック回帰
  ps_model = sm.Logit(y, X).fit()
  ps_score = ps_model.predict(X)

  # treatmentとps_scoreから成るデータフレーム
  all_df = pd.DataFrame({
      "treatment": y, 
      "ps_score": ps_score
  })

  #treatments = all_df["treatment"].unique()

  # all_dfをgroup1と2に分ける
  # treatment = 1をgroup1，treatment = 0をgroup2とする
  group1_df = all_df[all_df["treatment"] == 1].copy() # group1のサンプルを抽出
  group1_indices = group1_df.index                    # インデックスを取得
  group1_df = group1_df.reset_index(drop = True)      # インデックスを付け替える
  group2_df = all_df[all_df["treatment"] == 0].copy()
  group2_indices = group2_df.index
  group2_df = group2_df.reset_index(drop = True)

  # マッチングの閾値：傾向スコアの差が，傾向スコアの標準偏差の0.2倍より小さいサンプル同士をマッチさせる
  threshold = all_df["ps_score"].std() * 0.2

  # マッチしたサンプル格納用配列
  matched_group1_dfs = [] # group1でマッチしたサンプルのインデックス格納用
  matched_group2_dfs = [] # group2でマッチしたサンプルのインデックス格納用

  # マッチング用配列
  _group1_df = group1_df.copy()
  _group2_df = group2_df.copy()

  while True:
    
    # NearestNeighborsで最近傍点を一つ見つけ，マッチングする
    # group2の各サンプルについて，最も近いサンプルを一つgroup1から取ってくる
    # distances, indicesともに行数はgroup2のサンプル数．indicesにはマッチしたgroup1のindexが格納される．
    neigh = NearestNeighbors(n_neighbors = 1).fit(_group1_df["ps_score"].values.reshape(-1, 1))
    distances, indices = neigh.kneighbors(_group2_df["ps_score"].values.reshape(-1, 1))

    # 重複を削除する
    distance_df = pd.DataFrame({
        "distance": distances.reshape(-1), 
        "indices": indices.reshape(-1)
    })
    distance_df.index = _group2_df.index
    distance_df = distance_df.drop_duplicates(subset = "indices") # indicesが重複する行を削除

    # 閾値を超える行を削除
    distance_df = distance_df[distance_df["distance"] < threshold]

    # group1のサンプルすべてについてマッチングしたら終了
    if len(distance_df) == 0:
      break
    
    # マッチングしたサンプルを別配列に格納
    #group1_matched_indices = _group1_df.iloc[distance_df["indices"]].index.tolist()
    group1_matched_indices = _group1_df.iloc[distance_df["indices"]].index
    group2_matched_indices = distance_df.index
    matched_group1_dfs.append(_group1_df.loc[group1_matched_indices])
    matched_group2_dfs.append(_group2_df.loc[group2_matched_indices])

    # マッチングしたサンプルを削除
    _group1_df = _group1_df.drop(group1_matched_indices)
    _group2_df = _group2_df.drop(group2_matched_indices)

  # 付け替えたインデックスを元に戻す
  group1_df.index = group1_indices
  group2_df.index = group2_indices

  # それぞれのグループからマッチしたサンプルを抽出し，二つのグループを結合し，インデックスの昇順でソート
  matched_df = pd.concat([
                          group1_df.iloc[pd.concat(matched_group1_dfs).index], 
                          group2_df.iloc[pd.concat(matched_group2_dfs).index]
  ]).sort_index()
  matched_indices = matched_df.index # マッチしたサンプルのインデックス

  # マッチしたXとyを返す
  return X.loc[matched_indices], y.loc[matched_indices]

In [ ]:
# 最近傍マッチング実行
matchX, matchy = get_matched_dfs_using_propensity_score(x, y, random_state = 0)

Optimization terminated successfully.
         Current function value: 0.658837
         Iterations 5


In [ ]:
# マッチング後のデータで効果の推定
y = biased_data.loc[matchy.index].spend # 被説明変数：spend
x = sm.add_constant(matchy) # 説明変数：[const, treatment]
results = sm.OLS(y, x).fit()
print(results.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6134      0.138      4.433      0.000       0.342       0.885
treatment      0.7790      0.196      3.980      0.000       0.395       1.163


### dowhyライブラリを使用

- [参考]「効果検証入門 3章をPythonで書く」https://qiita.com/calderarie/items/2d631538ecf76d9e5096

In [ ]:
# ライブラリ
!pip install git+https://github.com/microsoft/dowhy
from dowhy import CausalModel

  Cloning https://github.com/microsoft/dowhy to /tmp/pip-req-build-rfela2b6
  Running command git clone -q https://github.com/microsoft/dowhy /tmp/pip-req-build-rfela2b6


In [ ]:
# インスタンス生成
biased_data = biased_data.astype({"treatment": "bool"}, copy = False) # 介入変数treatmentをbool化
model = CausalModel(data = biased_data, treatment = "treatment", outcome = "spend", common_causes = ["recency", "history", "channel"])

In [ ]:
# 最近傍マッチングによるATTの推定
identified_estimand = model.identify_effect(proceed_when_unidentifiable = True)
nearest_att = model.estimate_effect(identified_estimand, method_name = "backdoor.propensity_score_matching", target_units = "att")
nearest_ate = model.estimate_effect(identified_estimand, method_name="backdoor.propensity_score_matching", target_units='ate')
nearest_atc = model.estimate_effect(identified_estimand, method_name = "backdoor.propensity_score_matching", target_units = "atc")

# 結果
print(f"ATT: {nearest_att.value}")
print(f"ATE: {nearest_ate.value}")
print(f"ATC: {nearest_atc.value}")

ATT: 0.9176240680335502
ATE: 0.9249797963978066
ATC: 0.9335373043301475


## 3.2.2 逆確率重み付き推定